In [10]:
import numpy as np
import cv2

def calculate_orientation_hist(x, y, radius, gradient, angle, n, Sa):
    sigma = radius / 3
    radius_x_left = x-radius
    radius_x_right = x + radius
    radius_y_up = y - radius
    radius_y_down = y + radius

    sub_gradient = gradient[radius_y_up:radius_y_down+1, radius_x_left:radius_x_right+1]
    sub_angle = angle[radius_y_up:radius_y_down+1, radius_x_left:radius_x_right+1]

    X = np.arange(-(x - radius_x_left), (radius_x_right - x) + 1)
    Y = np.arange(-(y - radius_y_up), (radius_y_down - y) + 1)

    XX, YY = np.meshgrid(X, Y)

    gaussian_weight = np.exp(-(XX**2 + YY**2) / (2 * sigma**2))
    W1 = sub_gradient * gaussian_weight
    W = Sa[:W1.shape[0], :W1.shape[1]] * W1

    bin = np.round(sub_angle * n / 360).astype(int)
    bin[bin >= n] -= n
    bin[bin < 0] += n

    temp_hist = np.zeros(n)
    for i in range(n):
        wM = W[bin == i]
        if wM.size > 0:
            temp_hist[i] = np.sum(wM)

    hist = np.zeros(n)
    hist[0] = (temp_hist[n-2] + temp_hist[2]) / 16 + \
              4 * (temp_hist[n-1] + temp_hist[1]) / 16 + temp_hist[0] * 6 / 16
    hist[1] = (temp_hist[n-1] + temp_hist[3]) / 16 + \
              4 * (temp_hist[0] + temp_hist[2]) / 16 + temp_hist[1] * 6 / 16
    hist[2:n-2] = (temp_hist[0:n-4] + temp_hist[4:n]) / 16 + \
                  4 * (temp_hist[1:n-3] + temp_hist[3:n-1]) / 16 + temp_hist[2:n-2] * 6 / 16
    hist[n-2] = (temp_hist[n-4] + temp_hist[0]) / 16 + \
                4 * (temp_hist[n-3] + temp_hist[n-1]) / 16 + temp_hist[n-2] * 6 / 16
    hist[n-1] = (temp_hist[n-3] + temp_hist[1]) / 16 + \
                4 * (temp_hist[n-2] + temp_hist[0]) / 16 + temp_hist[n-1] * 6 / 16

    max_value = np.max(hist)
    return hist, max_value

In [11]:
import numpy as np
import scipy.io



# Load the MATLAB variables
matlab_data = scipy.io.loadmat('D:\Research Related\HasanHoca\RIFT2-multimodal-matching-rotation-python/third_party\RIFT2-multimodal-matching-rotation-master/matlab_variables.mat')

# Extract the variables
x = matlab_data['x'][0][0]
y = matlab_data['y'][0][0]
patch_size = matlab_data['patch_size'][0][0]
gradientImg = matlab_data['gradientImg']
gradientAng = matlab_data['gradientAng']
n = matlab_data['n'][0][0]
Sa = matlab_data['Sa']
matlab_hist = matlab_data['hist'][0]
matlab_max_value = matlab_data['max_value'][0][0]




# Run the Python function with the same inputs
python_hist, python_max_value = calculate_orientation_hist(x, y, patch_size / 2, gradientImg, gradientAng, n, Sa)

# Compare the results
print("MATLAB hist:", matlab_hist)
print("Python hist:", python_hist)
print("MATLAB max_value:", matlab_max_value)
print("Python max_value:", python_max_value)

# Check if they are close
hist_close = np.allclose(matlab_hist, python_hist, atol=1e-6)
max_value_close = np.isclose(matlab_max_value, python_max_value, atol=1e-6)

print("Histograms close:", hist_close)
print("Max values close:", max_value_close)


TypeError: slice indices must be integers or None or have an __index__ method